<a href="https://colab.research.google.com/github/Itsuki-Hamano123/auto_ml/blob/master/TPOT/tpot_regressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q tfds-nightly tensorflow matplotlib tpot
%pip show tpot

     |████████████████████████████████| 3.7MB 9.3MB/s 
     |████████████████████████████████| 92kB 8.9MB/s 
     |████████████████████████████████| 163kB 30.1MB/s 
Name: TPOT
Version: 0.11.6.post1
Summary: Tree-based Pipeline Optimization Tool
Home-page: https://github.com/EpistasisLab/tpot
Author: Randal S. Olson
Author-email: rso@randalolson.com
License: GNU/LGPLv3
Location: /usr/local/lib/python3.6/dist-packages
Requires: joblib, tqdm, scipy, scikit-learn, numpy, stopit, update-checker, deap, pandas
Required-by: 


In [2]:
import cloudpickle
import datetime
import os
import pprint

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow_datasets as tfds
from tpot import TPOTRegressor, config
from xgboost import XGBRegressor

## データセット読み込み
- radonデータセット[https://www.tensorflow.org/datasets/catalog/radon](https://www.tensorflow.org/datasets/catalog/radon)

In [3]:
def fetch_tf_dataset(data_name, 
                     split='train',
                     shuffle_files=True, as_supervised=True,
                     with_info=True, batch_size=None):
  '''TensorFlowデータセットからデータをフェッチ'''
  dataset, data_info = tfds.load(data_name, split=split,
                                  shuffle_files=shuffle_files,
                                  as_supervised=as_supervised,
                                  with_info=with_info,
                                  batch_size=batch_size)
  return dataset, data_info
  


data_name = 'radon' # @param{type:'string'}
x_name = 'features' # @param{type:'string'}
y_name = 'activity' # @param{type:'string'}
# インメモリでデータを読み込む
# TPOTがdatasetV1Adapter,generatorを受け付けないため
batch_size = -1

# データのフェッチ
tf_dataset, ds_info = fetch_tf_dataset(data_name=data_name,
                                     as_supervised=False,
                                     batch_size=batch_size)

display(ds_info)

Shuffling and writing examples to /root/tensorflow_datasets/radon/1.0.0.incomplete6QHRVM/radon-train.tfrecord


Dataset radon downloaded and prepared to /root/tensorflow_datasets/radon/1.0.0. Subsequent calls will reuse this data.


tfds.core.DatasetInfo(
    name='radon',
    version=1.0.0,
    description='Radon is a radioactive gas that enters homes through contact
points with the ground. It is a carcinogen that is the primary cause of lung
cancer in non-smokers. Radon levels vary greatly from household to household.
This dataset contains measured radon levels in U.S homes by county and state.
The 'activity' label is the measured radon concentration in pCi/L. Important
predictors are 'floor' (the floor of the house in which the measurement was
taken), 'county' (the U.S. county in which the house is located), and 'Uppm' (a
measurement of uranium level of the soil by county).',
    homepage='http://www.stat.columbia.edu/~gelman/arm/examples/radon/',
    features=FeaturesDict({
        'activity': tf.float32,
        'features': FeaturesDict({
            'Uppm': tf.float32,
            'adjwt': tf.float32,
            'basement': tf.string,
            'cntyfips': tf.int32,
            'county': tf.string,
      

### Dataframeに変換
TPOTがTFDS形式を受け付けないため

In [4]:
%%time

def convert_tfds_to_df(tf_ds, x_key, y_key):
  '''tf DatasetをDataframeに変換'''
  
  # yについて{key:numpy}形式のdictに変換
  y = {}
  y[y_key] = tf_ds[y_key].numpy()

  # xについて[{key:numpy},{key:numpy},,,]形式のdictに変換
  x = {}
  for f_key, element in tf_ds[x_key].items():
    if element.dtype == 'string':
      x[f_key] = _convert_byte_np_to_string_np(bytes_np=element.numpy())
    else:
      x[f_key] = element.numpy()
  
  y_df = pd.DataFrame.from_dict(y)
  x_df = pd.DataFrame.from_dict(x)
  df = pd.concat([y_df, x_df], axis=1)

  return df


def _convert_byte_np_to_string_np(bytes_np, decode_char='utf-8'):
  '''バイナリ文字列形式のnumpy配列を文字列形式のnumpy配列に変換'''
  decode_np = np.array([])
  for b in bytes_np:
    decode_np = np.append(decode_np, b.decode(decode_char))
  decode_np = decode_np.astype('object')
  return decode_np


# tf datasetからDataframeに変換
dataset = convert_tfds_to_df(tf_dataset, x_key=x_name, y_key=y_name)


display(dataset.head())
display(dataset.shape)


# カラム名の設定
y_col = y_name
x_cols = dataset.drop(columns=y_col).columns

display('y_col:{}'.format(y_col))
display('x_cols num:{}, names:{}'.format(len(x_cols), x_cols))

,activity,Uppm,adjwt,basement,cntyfips,county,dupflag,floor,idnum,lat,lon,pcterr,region,rep,room,startdt,starttm,state,state2,stfips,stopdt,stoptm,stratum,typebldg,wave,windoor,zip,zipflag
0,1.6,2.49759,292.219543,N,3,ALLEN,0,1,1681,41.091000,-85.067001,12.7,2,3,2,11489,2230,IN,IN,18,11689,2235,2,1,92,,46835,0
1,0.5,2.66527,364.958313,N,25,YAVAPAI,0,1,1457,34.599998,-112.554001,0.0,2,5,2,22688,833,AZ,AZ,4,22888,1010,2,1,56,,86325,0
2,3.0,2.43343,626.564575,Y,17,MIDDLESEX,0,0,4224,42.485001,-71.391998,9.7,6,1,2,32288,1000,MA,MA,25,32488,1000,1,1,23,,1778,0
3,0.6,2.43343,570.611755,N,17,MIDDLESEX,0,1,4094,42.485001,-71.391998,27.6,6,3,3,32488,700,MA,MA,25,32688,1200,1,1,62,,1432,0
4,1.2,2.10811,426.798859,Y,95,JACKSON,0,0,6849,39.007999,-94.347000,23.5,1,3,4,32288,605,MO,MO,29,32488,610,2,1,91,,64134,0


(12573, 28)

'y_col:activity'

"x_cols num:27, names:Index(['Uppm', 'adjwt', 'basement', 'cntyfips', 'county', 'dupflag', 'floor',\n       'idnum', 'lat', 'lon', 'pcterr', 'region', 'rep', 'room', 'startdt',\n       'starttm', 'state', 'state2', 'stfips', 'stopdt', 'stoptm', 'stratum',\n       'typebldg', 'wave', 'windoor', 'zip', 'zipflag'],\n      dtype='object')"

CPU times: user 10 s, sys: 337 ms, total: 10.4 s
Wall time: 10.4 s


## 文字列をone-hotエンコード
TPOTが文字列を受け付けないため

In [5]:
%%time

def one_hot_encode_df_val(df, target_col):
  '''文字列のカラムをone-hot-encodeしたカラムに置き換え
  WARNING : target_colはdropされる
  '''
  mlb = MultiLabelBinarizer()
  one_hot_feature = mlb.fit_transform([{str(val)} for val in df[target_col].values])
  insert_columns = [target_col+'_'+str(x+1) for x in range(len(one_hot_feature[0]))]
  insert_df = pd.DataFrame(one_hot_feature, columns=insert_columns)
  df = df.drop(columns=target_col)
  df = pd.concat([df, insert_df], axis=1)
  return df

target_cols = ['basement', 'county', 'state', 'state2', 'windoor']
for col in target_cols:
  dataset = one_hot_encode_df_val(dataset, col)
display(dataset.head())


# カラム名の設定
y_col = y_name
x_cols = dataset.drop(columns=y_col).columns

display('y_col:{}'.format(y_col))
display('x_cols num:{}, names:{}'.format(len(x_cols), x_cols))

,activity,Uppm,adjwt,cntyfips,dupflag,floor,idnum,lat,lon,pcterr,region,rep,room,startdt,starttm,stfips,stopdt,stoptm,stratum,typebldg,wave,zip,zipflag,basement_1,basement_2,basement_3,basement_4,county_1,county_2,county_3,county_4,county_5,county_6,county_7,county_8,county_9,county_10,county_11,county_12,county_13,...,county_365,county_366,county_367,county_368,county_369,county_370,county_371,county_372,county_373,county_374,county_375,county_376,county_377,county_378,county_379,county_380,county_381,county_382,county_383,county_384,county_385,county_386,state_1,state_2,state_3,state_4,state_5,state_6,state_7,state_8,state2_1,state2_2,state2_3,state2_4,state2_5,state2_6,state2_7,state2_8,state2_9,windoor_1
0,1.6,2.49759,292.219543,3,0,1,1681,41.091000,-85.067001,12.7,2,3,2,11489,2230,18,11689,2235,2,1,92,46835,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
1,0.5,2.66527,364.958313,25,0,1,1457,34.599998,-112.554001,0.0,2,5,2,22688,833,4,22888,1010,2,1,56,86325,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
2,3.0,2.43343,626.564575,17,0,0,4224,42.485001,-71.391998,9.7,6,1,2,32288,1000,25,32488,1000,1,1,23,1778,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
3,0.6,2.43343,570.611755,17,0,1,4094,42.485001,-71.391998,27.6,6,3,3,32488,700,25,32688,1200,1,1,62,1432,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,1.2,2.10811,426.798859,95,0,0,6849,39.007999,-94.347000,23.5,1,3,4,32288,605,29,32488,610,2,1,91,64134,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1


'y_col:activity'

"x_cols num:430, names:Index(['Uppm', 'adjwt', 'cntyfips', 'dupflag', 'floor', 'idnum', 'lat', 'lon',\n       'pcterr', 'region',\n       ...\n       'state2_1', 'state2_2', 'state2_3', 'state2_4', 'state2_5', 'state2_6',\n       'state2_7', 'state2_8', 'state2_9', 'windoor_1'],\n      dtype='object', length=430)"

CPU times: user 398 ms, sys: 100 ms, total: 498 ms
Wall time: 498 ms


## データ分割

In [6]:
SEED = 7 #@param{type:'number'}

X_train, X_test, y_train, y_test = train_test_split(dataset[x_cols].values, dataset[y_col].values,
                                                    test_size=0.2, random_state=SEED)

display(X_train.shape, y_train.shape)
display(X_test.shape, y_test.shape)

(10058, 430)

(10058,)

(2515, 430)

(2515,)

## TPOTで回帰モデル作成

In [7]:
GENERATION =  100#@param{type:'number'}
POPULATION =  100#@param{type:'number'}
CV =  5#@param{type:'number'}
EARLY_STOP_ROUND = 2 #@param:{type:'number'}
N_JOBS =  -1#@param{type:'number'}
VERBOSITY = 3 #@param{type:'number'}
SEARCH_STRATEGY = 'TPOT sparse'

pipeline_optimizer = TPOTRegressor(config_dict=SEARCH_STRATEGY,
                                   generations=GENERATION, # (default:100)
                                   population_size=POPULATION, # (default:100)
                                   cv=CV, #(default:5)
                                   early_stop=EARLY_STOP_ROUND,
                                   n_jobs=N_JOBS, # (default:1)
                                   random_state=SEED,
                                   verbosity=VERBOSITY,
                                   warm_start=True, # 以前のfit結果があれば続きから開始
                                   )

In [8]:
%%time
pipeline_optimizer.fit(X_train, y_train)

11 operators have been imported by TPOT.


Skipped pipeline #80 due to time out. Continuing to the next pipeline.
Skipped pipeline #90 due to time out. Continuing to the next pipeline.
_pre_test decorator: _random_mutation_operator: num_test=0 [01:17:01] /workspace/src/learner.cc:723: Check failed: mparam_.num_feature != 0 (0 vs. 0) : 0 feature is supplied.  Are you using raw Booster interface?
Stack trace:
  [bt] (0) /usr/local/lib/python3.6/dist-packages/xgboost/./lib/libxgboost.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x24) [0x7f028da1ecb4]
  [bt] (1) /usr/local/lib/python3.6/dist-packages/xgboost/./lib/libxgboost.so(xgboost::LearnerImpl::LazyInitModel()+0x5bf) [0x7f028dab41ef]
  [bt] (2) /usr/local/lib/python3.6/dist-packages/xgboost/./lib/libxgboost.so(XGBoosterUpdateOneIter+0x52) [0x7f028da1bac2]
  [bt] (3) /usr/lib/x86_64-linux-gnu/libffi.so.6(ffi_call_unix64+0x4c) [0x7f0339ac8dae]
  [bt] (4) /usr/lib/x86_64-linux-gnu/libffi.so.6(ffi_call+0x22f) [0x7f0339ac871f]
  [bt] (5) /usr/lib/python3.6/lib-dynload/_ctypes.cpyth

TPOTRegressor(config_dict='TPOT sparse', crossover_rate=0.1, cv=5,
              disable_update_check=False, early_stop=2, generations=100,
              log_file=None, max_eval_time_mins=5, max_time_mins=None,
              memory=None, mutation_rate=0.9, n_jobs=-1, offspring_size=None,
              periodic_checkpoint_folder=None, population_size=100,
              random_state=7, scoring=None, subsample=1.0, template=None,
              use_dask=False, verbosity=3, warm_start=True)

## パイプラインの出力

In [9]:
%cd /content/drive/My Drive/機械学習練習/AutoML/TPOT/radon_dataset
%ls

/content/drive/My Drive/機械学習練習/AutoML/TPOT/radon_dataset
export_pipeline/        radon-regressor_drop_country.ipynb  xgb_regressor.ipynb
Gitにpushしてないもの/  tpot_regressor.ipynb


In [10]:
%%time

EXPORT_DIR =  'export_pipeline' # @param {type:'string'}
file_prefix = 'radon-regressor_pipeline' # @param{type:'string'}

now = datetime.datetime.now().strftime('%Y%m%d_%H%m%s')
export_file = os.path.join(EXPORT_DIR, now+'_'+file_prefix)

# ベストなパイプラインのスクリプトファイル出力
pipeline_optimizer.export(output_file_name=export_file+'.py')


def dump_pkl(obj, path):
    '''objをpklファイルで出力'''
    with open(path, 'wb') as f:
        f.write(cloudpickle.dumps(obj))

# ベストなパイプラインをpklで出力
best_pipline = pipeline_optimizer.fitted_pipeline_
dump_pkl(best_pipline, export_file+'.pkl')

%ls {EXPORT_DIR}

20201128_09111606556683_radon-regressor_pipeline.pkl
20201128_09111606556683_radon-regressor_pipeline.py
CPU times: user 332 ms, sys: 775 ms, total: 1.11 s
Wall time: 1.73 s


## モデルの評価

### TPOTモデルクラスの評価関数

In [11]:
%%time
score = pipeline_optimizer.score(testing_features=X_test,
                                 testing_target=y_test)
display(score)

-17.177184655906

CPU times: user 74.6 ms, sys: 4.04 ms, total: 78.7 ms
Wall time: 78.8 ms


### sklearnの評価関数

In [13]:
%%time
def _calc_score(y_true, y_pred, sklearn_metric):
  '''sklearnの評価指標のスコアを計算'''
  score = sklearn_metric(y_true=y_true, y_pred=y_pred)
  return score


def evaluete_sk_metrics(y_true, y_pred, sklearn_metrics):
  '''sklearnの各種、評価関数を使用'''
  result = {}
  for sk_metric in sklearn_metrics:
    result[sk_metric.__name__] = _calc_score(y_true=y_true, y_pred=y_pred, sklearn_metric=sk_metric)
  return result


SK_METRICS=[r2_score, explained_variance_score, mean_squared_error]
eval_scores = {'train': {}, 'test': {}}

for key, X, y_true in zip(['train', 'test'], [X_train, X_test], [y_train, y_test]):
  y_pred = pipeline_optimizer.predict(X)
  eval_scores[key] = evaluete_sk_metrics(y_true=y_true, y_pred=y_pred, sklearn_metrics=SK_METRICS)
pprint.pprint(eval_scores)

{'test': {'explained_variance_score': 0.755174458026886,
          'mean_squared_error': 17.177183,
          'r2_score': 0.7551699598766066},
 'train': {'explained_variance_score': 0.8199043869972229,
           'mean_squared_error': 16.060017,
           'r2_score': 0.8198927551379287}}
CPU times: user 189 ms, sys: 2.89 ms, total: 192 ms
Wall time: 195 ms
